<a href="https://colab.research.google.com/github/NancyYiWang/WildFireSmokePrediction/blob/main/GEOSVisualizing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install netCDF4
!pip install matplotlib
!pip install cartopy

In [3]:
import os
from netCDF4 import Dataset

nc_directory = "/content/drive/My Drive/WildFire/DATA/NOAA_GOES_R"

nc_files = [f for f in os.listdir(nc_directory) if f.endswith('.nc')]

nc_file_path = os.path.join(nc_directory, nc_files[0])
print(f"\nReading file: {nc_file_path}")

nc_data = Dataset(nc_file_path, 'r')

print(nc_data)


Reading file: /content/drive/My Drive/WildFire/DATA/NOAA_GOES_R/ABI-L2-GFLSC-M6_v3r1_g16_s202401010001173_e202401010003546_c202401010005114.nc
<class 'netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.6
    Metadata_Conventions: CF-1.6, Unidata Dataset Discovery v1.0
    standard_name_vocabulary: CF Standard Name Table v76
    institution: DOC/NOAA/NESDIS/NDE > S-NPP Data Exploitation, NESDIS, NOAA, U.S. Department of Commerce
    naming_authority: gov.noaa.nesdis.nde
    processing_level: NOAA Level 2
    production_site: NSOF
    production_environment: OE
    sensor_band_identifier: 2, 7, 14
    sensor_band_central_radiation_wavelength: 0.64um, 3.9um, 11.2um
    satellite_name: GOES-16
    instrument_name: GOES-R Series Advanced Baseline Imager
    project: Enterprise Processing System: GOES-R Fog/Low Stratus Cloud Product
    summary: Fog Depth and Flight Rule Probabilities
    history: Enterprise FLS Algorithm Version 2.3.1
    references:

In [4]:
print("\nVariables in the file:")
for var in nc_data.variables:
    dimensions = nc_data.variables[var].dimensions
    print(f"{var}: {dimensions}")

    if not dimensions:
            value = nc_data.variables[var][:]
            print(f"Value: {value}")


Variables in the file:
StartRow: ()
Value: 1
StartColumn: ()
Value: 1
Latitude: ('Rows', 'Columns')
Longitude: ('Rows', 'Columns')
Fog_Lrc_Mask: ('Rows', 'Columns')
Fog_Qf: ('Rows', 'Columns', 'QfConst')
Fog_Qpi: ('Rows', 'Columns', 'QPIConst')
MVFR_Fog_Prob: ('Rows', 'Columns')
IFR_Fog_Prob: ('Rows', 'Columns')
IFR_RHonly_Fog_Prob: ('Rows', 'Columns')
LIFR_Fog_Prob: ('Rows', 'Columns')
Fog_Depth: ('Rows', 'Columns')
Sfc_Temp_Bias: ('Rows', 'Columns')
Sfc_Temp_Bias_Global: ('Rows', 'Columns')
Refl_Chn2_StdDev: ('Rows', 'Columns')
Refl_Chn2_StdDev_Lrc: ('Rows', 'Columns')
Refl_Chn2_Stddev_Composite: ('Rows', 'Columns')
BT_Chn14_StdDev: ('Rows', 'Columns')
Emiss_Chn7_Atm_Correc: ('Rows', 'Columns')
Emiss_Chn7_Composite: ('Rows', 'Columns')
Refl_Chn7_Composite: ('Rows', 'Columns')
Sfc_Elevation: ('Rows', 'Columns')
Sfc_RH: ('Rows', 'Columns')
Sfc_RH_Global: ('Rows', 'Columns')
Max_RH_500ft_Lay: ('Rows', 'Columns')
Max_RH_1000ft_Lay: ('Rows', 'Columns')
Max_RH_3000ft_Lay: ('Rows', 'Column

In [5]:
!pip install earthengine-api xarray rasterio folium -q

import ee
import os
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import rasterio
from rasterio.transform import from_origin
import folium
from PIL import Image

ee.Authenticate()

from google.colab import files
uploaded = files.upload()

nc_directory = "/content/drive/My Drive/WildFire/DATA/NOAA_GOES_R"
geo_tiff_directory = "/content/drive/My Drive/WildFire/DATA/GeoTIFF"
png_directory = "/content/drive/My Drive/WildFire/DATA/PNG"
os.makedirs(geo_tiff_directory, exist_ok=True)
os.makedirs(png_directory, exist_ok=True)

relevant_variables = [
    "Fog_Lrc_Mask",
    "MVFR_Fog_Prob",
    "IFR_Fog_Prob",
    "IFR_RHonly_Fog_Prob",
    "LIFR_Fog_Prob"
]

def convert_nc_to_geotiff(ds, variable, output_directory):
    try:
        data = ds[variable].values
        lon = ds["Longitude"].values
        lat = ds["Latitude"].values

        lon_min, lon_max = np.min(lon), np.max(lon)
        lat_min, lat_max = np.min(lat), np.max(lat)

        if data.shape != lat.shape or data.shape != lon.shape:
            print(f"Skipping variable '{variable}': Shape mismatch with Latitude and Longitude.")
            return None

        if np.isnan(data).all():
            print(f"Skipping variable '{variable}': Data is all NaN.")
            return None

        transform = from_origin(lon_min, lat_max, abs(lon[0, 1] - lon[0, 0]), abs(lat[1, 0] - lat[0, 0]))
        output_file = os.path.join(output_directory, f"{variable}.tif")

        with rasterio.open(
            output_file,
            "w",
            driver="GTiff",
            height=data.shape[0],
            width=data.shape[1],
            count=1,
            dtype=data.dtype.name if hasattr(data.dtype, "name") else "float32",
            crs="EPSG:4326",
            transform=transform,
        ) as dst:
            dst.write(data, 1)

        print(f"GeoTIFF saved at: {output_file}")
        return output_file
    except Exception as e:
        print(f"Error processing variable '{variable}': {e}")
        return None

def convert_geotiff_to_png(geo_tiff_path, output_directory):

    try:
        with rasterio.open(geo_tiff_path) as src:
            data = src.read(1)
            transform = src.transform
            bounds = src.bounds

        if np.isnan(data).all():
            print(f"Data in {geo_tiff_path} is all NaN. Skipping.")
            return None, None

        print(f"Processing {geo_tiff_path}: min={np.nanmin(data)}, max={np.nanmax(data)}, mean={np.nanmean(data)}")

        data = np.nan_to_num(data, nan=0)

        vmin, vmax = np.percentile(data, [5, 95])
        data_clipped = np.clip(data, vmin, vmax)

        data_normalized = ((data_clipped - vmin) / (vmax - vmin) * 255).astype("uint8")

        png_file_path = os.path.join(output_directory, os.path.splitext(os.path.basename(geo_tiff_path))[0] + ".png")
        Image.fromarray(data_normalized).save(png_file_path)

        print(f"Converted GeoTIFF to PNG: {png_file_path}")
        return png_file_path, bounds
    except Exception as e:
        print(f"Error converting GeoTIFF to PNG: {e}")
        return None, None

def visualize_layers_on_map(png_files):
    map_center = [0, 0]
    map = folium.Map(location=map_center, zoom_start=4)

    for variable, (png_file, bounds) in png_files.items():
        try:
            folium.raster_layers.ImageOverlay(
                name=variable,
                image=png_file,
                bounds=[[bounds.bottom, bounds.left], [bounds.top, bounds.right]],
                opacity=0.7,
            ).add_to(map)

            print(f"Added visualization for variable '{variable}'.")
        except Exception as e:
            print(f"Error visualizing variable '{variable}': {e}")

    folium.LayerControl().add_to(map)
    return map

nc_files = [f for f in os.listdir(nc_directory) if f.endswith('.nc')]
if not nc_files:
    print("No NetCDF files found in the directory.")
else:
    nc_file_path = os.path.join(nc_directory, nc_files[0])
    print(f"Processing file: {nc_file_path}")

    ds = xr.open_dataset(nc_file_path)
    two_d_variables = [var for var in ds.variables if ds[var].dims == ('Rows', 'Columns')]

    png_files = {}
    for variable in two_d_variables:
        if variable in relevant_variables:
            geo_tiff_file = convert_nc_to_geotiff(ds, variable, geo_tiff_directory)
            if geo_tiff_file:
                png_file, bounds = convert_geotiff_to_png(geo_tiff_file, png_directory)
                if png_file:
                    png_files[variable] = (png_file, bounds)

    if png_files:
        map = visualize_layers_on_map(png_files)
        map_path = '/content/drive/My Drive/WildFire/visualization_map.html'
        map.save(map_path)
        print(f"Map with relevant layers saved to: {map_path}")
        print(f"PNG files: {png_files}")
    else:
        print("No relevant GeoTIFF files were created.")

Saving wildfiresmokeprediction-d24859b8ae9a.json to wildfiresmokeprediction-d24859b8ae9a (24).json
Processing file: /content/drive/My Drive/WildFire/DATA/NOAA_GOES_R/ABI-L2-GFLSC-M6_v3r1_g16_s202401010001173_e202401010003546_c202401010005114.nc
GeoTIFF saved at: /content/drive/My Drive/WildFire/DATA/GeoTIFF/Fog_Lrc_Mask.tif
Processing /content/drive/My Drive/WildFire/DATA/GeoTIFF/Fog_Lrc_Mask.tif: min=0.0, max=1.0, mean=0.08794274181127548
Converted GeoTIFF to PNG: /content/drive/My Drive/WildFire/DATA/PNG/Fog_Lrc_Mask.png
GeoTIFF saved at: /content/drive/My Drive/WildFire/DATA/GeoTIFF/MVFR_Fog_Prob.tif
Processing /content/drive/My Drive/WildFire/DATA/GeoTIFF/MVFR_Fog_Prob.tif: min=0.0, max=99.0, mean=23.24563217163086
Converted GeoTIFF to PNG: /content/drive/My Drive/WildFire/DATA/PNG/MVFR_Fog_Prob.png
GeoTIFF saved at: /content/drive/My Drive/WildFire/DATA/GeoTIFF/IFR_Fog_Prob.tif
Processing /content/drive/My Drive/WildFire/DATA/GeoTIFF/IFR_Fog_Prob.tif: min=0.0, max=97.8925018310546